In [44]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tcn import TCN, tcn_full_summary

# Copy Memory

In this task, each input sequence has length T + 20. The first 10 values are chosen randomly among the digits 1, . . . , 8, with the rest being all zeros, except for the last 11 entries that are filled with the digit ‘9’ (the first ‘9’ is a delimiter). The goal is to generate an output of the same length that is zero everywhere except the last 10 values after the delimiter, where the model is expected to repeat the 10 values it encountered at the start of the input.

Ex.

Input  = 1234567876 000000000 99999999999 
         
         Message        T     Delimeter+Space

Output = 0000000000 000000000 01234567876

                              Message


In [45]:
def CreateCopyMemoryDataset(T, ENTRIES):
    """
    Inputs:
    T       (INT): The number of 0s between the message and delimeter
    ENTRIES (INT): The number of copy memory example entries to make
    
    Outputs:
    X       (np.array, shape:(Entries, T+20)): Copy memory input sequences
    y       (np.array, shape:(Entries, T+20)): Copy memoty output sequences
    """
    
    #Create random integer messages
    RandIntMessages = np.random.randint(low=1, high=8, size=((ENTRIES, 10)))
    
    #Create delimeter
    Delimeter = np.zeros((ENTRIES, 11)) + 9
    
    #Create spacer
    Spacer = np.zeros((ENTRIES, T-1))
    
    #Create X
    # [Message, Spacer, Delimeter]
    X = np.hstack((RandIntMessages, Spacer, Delimeter))
    
    #Create empty array for y,then transplant the X messages to the correct location
    y = np.zeros((ENTRIES, T+20))
    y [:, -10:] = X[:,:10]
        
    return X, y

In [46]:
X, y = CreateCopyMemoryDataset(3, 10)
print(X.shape)
print(y.shape)

print(X[0])
print(y[0])

(10, 23)
(10, 23)
[7. 3. 3. 1. 2. 7. 2. 6. 6. 6. 0. 0. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7. 3. 3. 1. 2. 7. 2. 6. 6. 6.]


# TCN

Implmenting TCN using: ["https://github.com/philipperemy/keras-tcn"]



In [87]:
def buildTCNModel(T):

    #Setup TCN architecture
    tcn_layer = TCN(
        input_shape =(T+20,1),
        nb_filters=64,
        kernel_size=4,
        nb_stacks=1,
        dilations=(1, 2, 4, 8, 16, 32),
        padding='causal',
        use_skip_connections=True,
        dropout_rate=0.1,
        return_sequences=True,
        activation='relu',
        kernel_initializer='he_normal',
        use_batch_norm=False,
        use_layer_norm=False,
        use_weight_norm=True
    )

    #Create a Tensorflow model
    TCNModel = tf.keras.models.Sequential([tcn_layer, tf.keras.layers.Dense(1)])
    
    #Setup model
    TCNModel.compile(optimizer="adam", loss="mse")
    
    return TCNModel

# Training and Testing

In [90]:
#Define Copy Memory Problem
T = 20

#Create Training Examples
ENTRIES = 10000
X, y = CreateCopyMemoryDataset(T, ENTRIES)
print(X.shape)
print(y.shape)

#Build Model
TCNModel = buildTCNModel(T)

(10000, 40)
(10000, 40)


In [91]:
history = TCNModel.fit(X, y, epochs = 50,
                      batch_size = 256, steps_per_epoch = 16,
                      validation_split=0.2)

Epoch 1/50
16/16 [==============================] - 13s 503ms/step - loss: 3.9320 - val_loss: 1.5195
Epoch 2/50
16/16 [==============================] - 6s 370ms/step - loss: 1.7217 - val_loss: 1.0797
Epoch 3/50
16/16 [==============================] - 6s 388ms/step - loss: 1.3649 - val_loss: 0.9073
Epoch 4/50
16/16 [==============================] - 6s 368ms/step - loss: 1.1576 - val_loss: 0.6812
Epoch 5/50
16/16 [==============================] - 6s 386ms/step - loss: 0.8404 - val_loss: 0.2188
Epoch 6/50
16/16 [==============================] - 6s 367ms/step - loss: 0.5059 - val_loss: 0.0730
Epoch 7/50
16/16 [==============================] - 6s 390ms/step - loss: 0.3956 - val_loss: 0.0490
Epoch 8/50
16/16 [==============================] - 6s 376ms/step - loss: 0.3384 - val_loss: 0.0356
Epoch 9/50
16/16 [==============================] - 6s 385ms/step - loss: 0.3017 - val_loss: 0.0322
Epoch 10/50
16/16 [==============================] - 6s 369ms/step - loss: 0.2748 - val_loss: 0.031

In [118]:
X, y = CreateCopyMemoryDataset(T, 1)
pred_y = TCNModel.predict(X)

print(y[:,-10:])
print(np.round(pred_y,2).reshape(1,-1)[:,-10:])
print(np.round(pred_y).reshape(1,-1)[:,-10:])

[[4. 5. 7. 2. 4. 5. 3. 1. 5. 1.]]
[[3.95 4.84 6.61 1.62 3.89 4.92 2.83 0.8  4.81 0.97]]
[[4. 5. 7. 2. 4. 5. 3. 1. 5. 1.]]
